## **NLP - Text Classification Project**
Group H - August 2025

Classification of tweets from Twitter that have been manually tagged for sentiment analysis.

In [1]:
# # Needed for Google Colab
# !pip install --quiet evaluate transformers optuna datasets nltk scikit-learn
# !pip install numpy==1.26.4

In [2]:
%env CUDA_LAUNCH_BLOCKING=1

from wordcloud import WordCloud, STOPWORDS
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from tqdm import tqdm
from collections import Counter
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import Dataset
from torch.utils.data import DataLoader, TensorDataset
from optuna.pruners import MedianPruner
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import os
import re
import string
import time
import glob
import nltk
import evaluate
import transformers
import torch
import optuna
import wandb
wandb.login()
# API key - 0cbd7fe3cffd71df993b30edb4fa0db94f114413

os.environ["WANDB_PROJECT"] = "tweet-sentiment-classification_split_to_test_maxl_256"
os.environ["WANDB_INIT_TIMEOUT"] = "180"

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

env: CUDA_LAUNCH_BLOCKING=1
Mounted at /content/drive


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hillas (hillas-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
# Load data
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp_project/Data/Corona_NLP_train.csv', encoding='latin1')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp_project/Data/Corona_NLP_test.csv', encoding='latin1')

In [4]:
# Merge and shuffle for better stratified splits
df_full = pd.concat([df_train, df_test], ignore_index=True)
df_full = df_full.sample(frac=1.0, random_state=42).reset_index(drop=True)

### Pre-processing the Data

The tweets were cleaned by lowercasing (to reduce redundancy), removing stopwords, punctuation, numbers, short words, and applying lemmatization to reduce words to their base form (e.g. running → run). <br>This helps reduce noise and improve model performance.

In [5]:
# Try without pre-processing
is_preprocessed = "no_preprocess"
df_full["clean_text"] = df_full["OriginalTweet"]

In [6]:
# # Try minimal pre-processing
# def light_preprocess(text):
#     return text.strip()                             # Remove unnecessary spaces

# is_preprocessed = "minimal_preprocess"
# df_full["clean_text"] = df_full["OriginalTweet"].apply(light_preprocess)

In [7]:
# # Try pre-processing
# def clean_text(text):
#     text = str(text).lower()
#     text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # URLs
#     text = re.sub(r'\@\w+|\#','', text)  # Mentions & hashtags
#     text = re.sub(r'\n', ' ', text)  # Line breaks
#     text = re.sub(r"[^a-zA-Z']", ' ', text)  # Keep letters only
#     text = re.sub(r'\s+', ' ', text).strip()  # Extra whitespace
#     return text

# is_preprocessed = "w_preprocess"
# df_full["clean_text"] = df_full["OriginalTweet"].apply(clean_text)

## Fine-Tuning Pretrained Language Models

Apply NLP techniques using transfer learning on our tweets dataset. Specifically, fine-tuning two pretrained transformer-based models from the Hugging Face library — BERT and RoBERTa — on our sentiment classification task. These models will be trained using both standard PyTorch and the Hugging Face API. Model performance will be monitored and tuned using hyperparameter optimization (Optuna) and experiment tracking (Weights & Biases).

**Load Pretrained Models**

Initialize tokenizers and models for BERT and RoBERTa, both widely used transformer architectures for text classification. The classification head is configured based on the number of sentiment labels.

In [8]:
# Choose pretrained models
bert_model_name = "bert-base-uncased"
roberta_model_name = "roberta-base"

sentiment_labels = df_full['Sentiment'].unique()
n_labels = len(sentiment_labels)

# Load BERT tokenizer and model
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name, num_labels=n_labels)

# Load RoBERTa tokenizer and model
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_model_name, num_labels=n_labels)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Encode Sentiment Labels**

Map each unique sentiment label to a numeric ID for model compatibility, and apply this mapping to both training and validation datasets.

In [9]:
# Mapping sentiments to unique numeric IDs
unique_labels = sorted(df_full["Sentiment"].unique())
label2id = {label: idx for idx, label in enumerate(unique_labels)}
df_full["label"] = df_full["Sentiment"].map(label2id)

In [10]:
# Stratified split: 70% train, 15% val, 15% test
train_val_df, test_df = train_test_split(df_full, test_size=0.15, stratify=df_full["label"], random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.1765, stratify=train_val_df["label"], random_state=42)

# Confirm sizes
print("Train size:", len(train_df))
print("Val size:", len(val_df))
print("Test size:", len(test_df))

Train size: 31466
Val size: 6745
Test size: 6744


**Use Small Subsets for Quick Evaluation**

Select shuffled samples from each training and validation dataset for both BERT and RoBERTa. This allows faster experimentation during model development.

In [11]:
train_subset_df, _ = train_test_split(
    train_df[["clean_text", "label"]],
    train_size=2000,
    stratify=train_df["label"],
    random_state=42
)

val_subset_df, _ = train_test_split(
    val_df[["clean_text", "label"]],
    train_size=500,
    stratify=val_df["label"],
    random_state=42
)

**Tokenization**

Define tokenization functions for BERT and RoBERTa to preprocess text with truncation and fixed padding.

Transform training and validation DataFrames into Dataset objects compatible with Hugging Face workflows.

Apply tokenization to training and validation datasets using each model's tokenizer.

In [12]:
# Tokenize function
def tokenize_function_bert(examples):
    return bert_tokenizer(examples["clean_text"], truncation=True, padding='max_length', max_length=256)

def tokenize_function_roberta(examples):
    return roberta_tokenizer(examples["clean_text"], truncation=True, padding='max_length', max_length=256)

In [13]:
# Convert DataFrame to Hugging Face Dataset
hf_subset_train = Dataset.from_pandas(train_subset_df)
hf_subset_val = Dataset.from_pandas(val_subset_df)

hf_train = Dataset.from_pandas(train_df[["clean_text", "label"]])
hf_val = Dataset.from_pandas(val_df[["clean_text", "label"]])
hf_test = Dataset.from_pandas(test_df[["clean_text", "label"]])

In [14]:
# Tokenize subsets
# Tokenize for BERT
tokenized_bert_train_sub = hf_subset_train.map(tokenize_function_bert, batched=True)
tokenized_bert_train_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_bert_val_sub = hf_subset_val.map(tokenize_function_bert, batched=True)
tokenized_bert_val_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Tokenize for RoBERTa
tokenized_roberta_train_sub = hf_subset_train.map(tokenize_function_roberta, batched=True)
tokenized_roberta_train_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_roberta_val_sub = hf_subset_val.map(tokenize_function_roberta, batched=True)
tokenized_roberta_val_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [15]:
# Tokenize full dataset
# Tokenize for BERT
tokenized_bert_train = hf_train.map(tokenize_function_bert, batched=True)
tokenized_bert_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_bert_val = hf_val.map(tokenize_function_bert, batched=True)
tokenized_bert_val.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_bert_test = hf_test.map(tokenize_function_bert, batched=True)
tokenized_bert_test.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Tokenize for RoBERTa
tokenized_roberta_train = hf_train.map(tokenize_function_roberta, batched=True)
tokenized_roberta_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_roberta_val = hf_val.map(tokenize_function_roberta, batched=True)
tokenized_roberta_val.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_roberta_test = hf_test.map(tokenize_function_roberta, batched=True)
tokenized_roberta_test.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/31466 [00:00<?, ? examples/s]

Map:   0%|          | 0/6745 [00:00<?, ? examples/s]

Map:   0%|          | 0/6744 [00:00<?, ? examples/s]

Map:   0%|          | 0/31466 [00:00<?, ? examples/s]

Map:   0%|          | 0/6745 [00:00<?, ? examples/s]

Map:   0%|          | 0/6744 [00:00<?, ? examples/s]

In [16]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "f1_macro": f1_metric.compute(predictions=predictions, references=labels, average="macro")["f1"]
    }

**Hyperparameters Tuning with Optuna**

In [17]:
def build_trainer(model_checkpoint, trial, run_prefix, train_dataset, val_dataset):
    # Sample hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    num_epochs = trial.suggest_int("num_train_epochs", 2, 5)
    n_samples = len(train_dataset)

    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=5)
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    run_name = f"{run_prefix}-ep{num_epochs}-lr{learning_rate}-bs{batch_size}-samples{n_samples}-run{int(time.time())}-{is_preprocessed}"

    args = TrainingArguments(
        output_dir=f"./results/{run_prefix}/{run_name}",
        disable_tqdm=True,
        fp16=True,
        eval_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        learning_rate=learning_rate,
        weight_decay=0.01,
        label_smoothing_factor=0.1,
        load_best_model_at_end=True,
        save_total_limit=1,
        logging_strategy="epoch",
        logging_dir=f"./logs/{run_prefix}/{run_name}",
        run_name=run_name,
        report_to="wandb",
        metric_for_best_model="f1_macro",
        greater_is_better=True
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    return trainer

In [18]:
def objective_bert(trial):
    trainer = build_trainer(
        model_checkpoint="bert-base-uncased",
        trial=trial,
        run_prefix="bert",
        train_dataset=tokenized_bert_train_sub,
        val_dataset=tokenized_bert_val_sub
    )
    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_f1_macro"]

def objective_roberta(trial):
    trainer = build_trainer(
        model_checkpoint="roberta-base",
        trial=trial,
        run_prefix="roberta",
        train_dataset=tokenized_roberta_train_sub,
        val_dataset=tokenized_roberta_val_sub
    )
    trainer.train()
    eval_result = trainer.evaluate()

    return eval_result["eval_f1_macro"]

In [19]:
study_bert = optuna.create_study(direction="maximize",
                                 pruner=MedianPruner(n_startup_trials=2, n_warmup_steps=1),
                                 study_name=f"bert_study_stratify_{is_preprocessed}",
                                 storage=f"sqlite:////content/drive/MyDrive/Colab Notebooks/nlp_project/optuna/bert_study_stratify_maxl_256_{is_preprocessed}.db",
                                 load_if_exists=True)
study_bert.optimize(objective_bert, n_trials=5)
wandb.finish()

[I 2025-08-07 20:59:54,986] Using an existing study with name 'bert_study_stratify_no_preprocess' instead of creating a new one.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-523513336.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.5293, 'grad_norm': 3.855067491531372, 'learning_rate': 2.1647282149400416e-05, 'epoch': 1.0}
{'eval_loss': 1.3812739849090576, 'eval_accuracy': 0.42, 'eval_f1_macro': 0.343830610238052, 'eval_runtime': 1.0131, 'eval_samples_per_second': 493.514, 'eval_steps_per_second': 31.585, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.2121, 'grad_norm': 7.0508599281311035, 'learning_rate': 1.625702265004772e-05, 'epoch': 2.0}
{'eval_loss': 1.2129623889923096, 'eval_accuracy': 0.532, 'eval_f1_macro': 0.540160959732994, 'eval_runtime': 0.9841, 'eval_samples_per_second': 508.06, 'eval_steps_per_second': 32.516, 'epoch': 2.0}


...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.8072, 'grad_norm': 7.888729572296143, 'learning_rate': 3.846884725003771e-07, 'epoch': 3.0}
{'eval_loss': 1.202750325202942, 'eval_accuracy': 0.578, 'eval_f1_macro': 0.5760127646396417, 'eval_runtime': 1.0009, 'eval_samples_per_second': 499.574, 'eval_steps_per_second': 31.973, 'epoch': 3.0}
{'train_runtime': 47.5808, 'train_samples_per_second': 126.101, 'train_steps_per_second': 7.881, 'train_loss': 1.1215946248372395, 'epoch': 3.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
[I 2025-08-07 21:04:53,813] Trial 9 finished with value: 0.5760127646396417 and parameters: {'learning_rate': 4.808605906254714e-05, 'batch_size': 16, 'num_train_epochs': 3}. Best is trial 7 with value: 0.5938229373061356.


{'eval_loss': 1.202750325202942, 'eval_accuracy': 0.578, 'eval_f1_macro': 0.5760127646396417, 'eval_runtime': 1.032, 'eval_samples_per_second': 484.517, 'eval_steps_per_second': 31.009, 'epoch': 3.0}


eval/accuracy,▄▇▆▇▇▇▃▃▃▇▇██▁▂▄▅▅▅▅▇██
eval/f1_macro,▄▇▇█▇█▃▄▄▇▇██▁▂▅▆▆▆▅▇██
eval/loss,▅▁▂▃▃▃▆▅▅▁▁▂▂█▇▅▄▃▃▂▁▁▁
eval/runtime,▃▃▃▃▃▃▁▁▁▇▇██▁▁▁▁▁▁▃▄▃▄
eval/samples_per_second,▄▄▄▄▄▄██▇▁▁▁▁█████▇▄▄▄▄
eval/steps_per_second,▅▅▅▅▅▅▂▂▁███▇▂▂▂▂▂▁▅▅▅▅
train/epoch,▁▁▃▃▅▅▆████▁▁▃▃▃▁▁▃▃▅▅▅▁▁▃▃▅▆▆███▁▁▃▃▅▅▅
train/global_step,▂▂▃▃▄▄▅▇▇▇▇▁▁▂▂▂▃▃▅▅███▁▁▂▂▂▃▃▄▄▄▂▂▃▃▄▄▄
train/grad_norm,▁▄▅▇█▂▃▅▅▆▁▂▅▄▄▃▂▄
train/learning_rate,▆▅▃▂▁▄▁▇▄▁▃▃▂▂▁█▅▁
train/loss,█▅▃▂▁█▇▇▄▂██▇▆▅▇▄▂


In [20]:
study_roberta = optuna.create_study(direction="maximize",
                                    pruner=MedianPruner(n_startup_trials=2, n_warmup_steps=1),
                                    study_name=f"roberta_study_stratify_{is_preprocessed}",
                                    storage=f"sqlite:////content/drive/MyDrive/Colab Notebooks/nlp_project/optuna/roberta_study_stratify_maxl_256_{is_preprocessed}.db",
                                    load_if_exists=True)
study_roberta.optimize(objective_roberta, n_trials=5)
wandb.finish()

[I 2025-08-07 21:05:27,640] Using an existing study with name 'roberta_study_stratify_no_preprocess' instead of creating a new one.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-523513336.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.5538, 'grad_norm': 7.124143123626709, 'learning_rate': 1.6591987912395855e-05, 'epoch': 1.0}
{'eval_loss': 1.4615013599395752, 'eval_accuracy': 0.374, 'eval_f1_macro': 0.2645378140912062, 'eval_runtime': 0.9953, 'eval_samples_per_second': 502.362, 'eval_steps_per_second': 32.151, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.2901, 'grad_norm': 17.27823257446289, 'learning_rate': 1.249356858742158e-05, 'epoch': 2.0}
{'eval_loss': 1.3017396926879883, 'eval_accuracy': 0.468, 'eval_f1_macro': 0.47369367913892046, 'eval_runtime': 0.9964, 'eval_samples_per_second': 501.797, 'eval_steps_per_second': 32.115, 'epoch': 2.0}


...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.1293, 'grad_norm': 11.650919914245605, 'learning_rate': 2.3275669641139022e-07, 'epoch': 3.0}
{'eval_loss': 1.285720944404602, 'eval_accuracy': 0.496, 'eval_f1_macro': 0.5099192149439827, 'eval_runtime': 0.9895, 'eval_samples_per_second': 505.304, 'eval_steps_per_second': 32.339, 'epoch': 3.0}
{'train_runtime': 49.0209, 'train_samples_per_second': 122.397, 'train_steps_per_second': 7.65, 'train_loss': 1.32797998046875, 'epoch': 3.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
[I 2025-08-07 21:12:32,054] Trial 9 finished with value: 0.5099192149439827 and parameters: {'learning_rate': 2.1820940288567834e-05, 'batch_size': 16, 'num_train_epochs': 3}. Best is trial 4 with value: 0.5787705726512649.


{'eval_loss': 1.285720944404602, 'eval_accuracy': 0.496, 'eval_f1_macro': 0.5099192149439827, 'eval_runtime': 1.0443, 'eval_samples_per_second': 478.769, 'eval_steps_per_second': 30.641, 'epoch': 3.0}


eval/accuracy,▃▆▇█▇█▁▃▅▆▆▆▅▇▇███▄▆▇███▃▆▇▇
eval/f1_macro,▃▇████▁▄▆▇▇▇▅▇▇███▄▇▇███▃▇▇▇
eval/loss,▆▂▁▂▂▂█▅▃▂▂▂▃▂▂▂▃▃▄▂▁▃▃▃▅▂▂▂
eval/runtime,▃▃▄▃▃▃▁▁▁▁▁▁▃▃▃▃▃▄██████▃▃▃▄
eval/samples_per_second,▄▄▄▄▄▄█████▇▄▄▄▄▄▃▁▁▁▁▁▁▄▄▄▄
eval/steps_per_second,▅▅▅▆▆▅▂▂▂▂▂▁▅▅▅▅▅▄█▇████▅▅▅▅
train/epoch,▁▃▃▅▅██▁▁▃▆▆███▁▃▃▅▅▆███▁▃▃▅▅▆████▁▃▃▅▅▅
train/global_step,▁▁▂▂▃▃▄▄▄▄▁▁▁▁▂▂▂▂▂▁▂▂▃▃▃▄▂▂▃▃▅▆▆███▁▃▃▃
train/grad_norm,▂▃▂▄▄▁▃▄▃▃▂▃▃▃▂▃▃▃█▆▂▃▂
train/learning_rate,▆▅▄▂▁▄▄▃▂▁█▆▅▃▁▄▃▂▂▁▅▃▁
train/loss,█▆▄▃▂█▇▆▅▄█▅▄▂▁█▆▄▃▂█▆▄


In [21]:
best_trial_bert = study_bert.best_trial
print('Bert best trial on subset:')
print(best_trial_bert.params)
best_trial_roberta = study_roberta.best_trial
print('RoBerta best trial on subset:')
print(best_trial_roberta.params)

Bert best trial on subset:
{'learning_rate': 4.4215134189001845e-05, 'batch_size': 8, 'num_train_epochs': 3}
RoBerta best trial on subset:
{'learning_rate': 3.9023135920177784e-05, 'batch_size': 16, 'num_train_epochs': 5}


In [22]:
best_params_bert = best_trial_bert.params
run_name_bert = f"bert_final_stratify_{is_preprocessed}-ep{best_params_bert['num_train_epochs']}-lr{best_params_bert['learning_rate']:.1e}-bs{best_params_bert['batch_size']}"
wandb.init(project="tweet-sentiment-classification_split_to_test_maxl_256", name=run_name_bert, reinit=True)

final_trainer_bert = build_trainer(
    model_checkpoint="bert-base-uncased",
    trial=best_trial_bert,
    run_prefix=f"bert_final_stratify_{is_preprocessed}",
    train_dataset=tokenized_bert_train,
    val_dataset=tokenized_bert_val
)
final_trainer_bert.train()
final_trainer_bert.evaluate(tokenized_bert_test)
wandb.finish()

wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-523513336.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.949, 'grad_norm': 20.738698959350586, 'learning_rate': 2.9495488177428533e-05, 'epoch': 1.0}
{'eval_loss': 0.7834368348121643, 'eval_accuracy': 0.8105263157894737, 'eval_f1_macro': 0.8174129839043711, 'eval_runtime': 22.2296, 'eval_samples_per_second': 303.424, 'eval_steps_per_second': 37.967, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.6851, 'grad_norm': 2.0032002925872803, 'learning_rate': 1.4764602934998836e-05, 'epoch': 2.0}
{'eval_loss': 0.699436366558075, 'eval_accuracy': 0.8613787991104522, 'eval_f1_macro': 0.8665421915197182, 'eval_runtime': 21.9741, 'eval_samples_per_second': 306.952, 'eval_steps_per_second': 38.409, 'epoch': 2.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.5624, 'grad_norm': 0.16492992639541626, 'learning_rate': 3.371769256914223e-08, 'epoch': 3.0}
{'eval_loss': 0.7147858142852783, 'eval_accuracy': 0.8744255003706449, 'eval_f1_macro': 0.8788204680910514, 'eval_runtime': 21.9778, 'eval_samples_per_second': 306.901, 'eval_steps_per_second': 38.402, 'epoch': 3.0}
{'train_runtime': 1136.6259, 'train_samples_per_second': 83.051, 'train_steps_per_second': 10.383, 'train_loss': 0.7321671843225724, 'epoch': 3.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 0.7161492109298706, 'eval_accuracy': 0.8733689205219455, 'eval_f1_macro': 0.8769232968000985, 'eval_runtime': 22.1349, 'eval_samples_per_second': 304.678, 'eval_steps_per_second': 38.085, 'epoch': 3.0}


eval/accuracy,▁▇██
eval/f1_macro,▁▇██
eval/loss,█▁▂▂
eval/runtime,█▁▁▅
eval/samples_per_second,▁██▃
eval/steps_per_second,▁██▃
train/epoch,▁▁▅▅████
train/global_step,▁▁▅▅████
train/grad_norm,█▂▁
train/learning_rate,█▅▁
train/loss,█▃▁


In [23]:
best_params_roberta = best_trial_roberta.params
run_name_roberta = f"roberta_final_stratify_{is_preprocessed}-ep{best_params_roberta['num_train_epochs']}-lr{best_params_roberta['learning_rate']:.1e}-bs{best_params_roberta['batch_size']}"
wandb.init(project="tweet-sentiment-classification_split_to_test_maxl_256", name=run_name_roberta, reinit=True)

final_trainer_roberta = build_trainer(
    model_checkpoint="roberta-base",
    trial=best_trial_roberta,
    run_prefix=f"roberta_final_stratify_{is_preprocessed}",
    train_dataset=tokenized_roberta_train,
    val_dataset=tokenized_roberta_val
)
final_trainer_roberta.train()
final_trainer_roberta.evaluate(tokenized_roberta_test)
wandb.finish()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-523513336.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.1045, 'grad_norm': 15.890998840332031, 'learning_rate': 3.124231542811183e-05, 'epoch': 1.0}
{'eval_loss': 0.8863462209701538, 'eval_accuracy': 0.7405485544848036, 'eval_f1_macro': 0.7507302485192145, 'eval_runtime': 13.7521, 'eval_samples_per_second': 490.47, 'eval_steps_per_second': 30.686, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.7892, 'grad_norm': 4.179565906524658, 'learning_rate': 2.344165602607121e-05, 'epoch': 2.0}
{'eval_loss': 0.804499626159668, 'eval_accuracy': 0.792290585618977, 'eval_f1_macro': 0.8006433994853828, 'eval_runtime': 13.3374, 'eval_samples_per_second': 505.722, 'eval_steps_per_second': 31.64, 'epoch': 2.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.5203, 'grad_norm': 0.5141141414642334, 'learning_rate': 3.174225595947354e-08, 'epoch': 5.0}
{'eval_loss': 0.8042707443237305, 'eval_accuracy': 0.8465530022238695, 'eval_f1_macro': 0.8515113328830293, 'eval_runtime': 13.369, 'eval_samples_per_second': 504.527, 'eval_steps_per_second': 31.566, 'epoch': 5.0}
{'train_runtime': 1096.9863, 'train_samples_per_second': 143.42, 'train_steps_per_second': 8.965, 'train_loss': 0.7311791284713396, 'epoch': 5.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 0.8232207298278809, 'eval_accuracy': 0.8404507710557533, 'eval_f1_macro': 0.8450049045492734, 'eval_runtime': 13.3989, 'eval_samples_per_second': 503.325, 'eval_steps_per_second': 31.495, 'epoch': 5.0}


eval/accuracy,▁▄▅███
eval/f1_macro,▁▄▄███
eval/loss,█▃▅▁▃▄
eval/runtime,█▁▁▁▂▂
eval/samples_per_second,▁███▇▇
eval/steps_per_second,▁███▇▇
train/epoch,▁▁▃▃▅▅▆▆████
train/global_step,▁▁▃▃▅▅▆▆████
train/grad_norm,█▃█▅▁
train/learning_rate,█▆▅▃▁
train/loss,█▄▃▂▁


In [24]:
final_trainer_bert.save_model(f"models/bert_final_stratify_{is_preprocessed}")
bert_tokenizer.save_pretrained(f"models/bert_final_stratify_{is_preprocessed}")
!cp -r models/bert_final_stratify_{is_preprocessed} "/content/drive/MyDrive/Colab Notebooks/nlp_project/models/w_test_split/bert_best_model_stratify_maxl_256_{is_preprocessed}"

final_trainer_roberta.save_model(f"models/roberta_final_stratify_{is_preprocessed}")
roberta_tokenizer.save_pretrained(f"models/roberta_final_stratify_{is_preprocessed}")
!cp -r models/roberta_final_stratify_{is_preprocessed} "/content/drive/MyDrive/Colab Notebooks/nlp_project/models/w_test_split/roberta_best_model_stratify_maxl_256_{is_preprocessed}"

#### Ensemble Bert and RoBerta

Load best models

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

best_bert_path = f"/content/drive/MyDrive/Colab Notebooks/nlp_project/models/w_test_split/bert_best_model_stratify_maxl_256_{is_preprocessed}"
bert_model = AutoModelForSequenceClassification.from_pretrained(best_bert_path).to(device)
bert_tokenizer = AutoTokenizer.from_pretrained(best_bert_path)

best_roberta_path = f"/content/drive/MyDrive/Colab Notebooks/nlp_project/models/w_test_split/roberta_best_model_stratify_maxl_256_{is_preprocessed}"
roberta_model = AutoModelForSequenceClassification.from_pretrained(best_roberta_path).to(device)
roberta_tokenizer = AutoTokenizer.from_pretrained(best_roberta_path)


Tokenize testset

In [26]:
bert_inputs = bert_tokenizer(
    list(test_df["clean_text"]),
    padding=True,
    truncation=True,
    return_tensors="pt"
).to(device)

roberta_inputs = roberta_tokenizer(
    list(test_df["clean_text"]),
    padding=True,
    truncation=True,
    return_tensors="pt"
).to(device)

Generate predictions with both models

In [27]:
# Prepare DataLoader
batch_size = 32  # adjust if needed

bert_dataset = TensorDataset(
    bert_inputs["input_ids"],
    bert_inputs["attention_mask"]
)
bert_loader = DataLoader(bert_dataset, batch_size=batch_size)

roberta_dataset = TensorDataset(
    roberta_inputs["input_ids"],
    roberta_inputs["attention_mask"]
)
roberta_loader = DataLoader(roberta_dataset, batch_size=batch_size)

In [28]:
bert_model.eval()
roberta_model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

Ensemble probabilities

In [29]:
bert_probs_all = []
roberta_probs_all = []

In [30]:
with torch.no_grad():
    for (bert_input_ids, bert_attention_mask), (roberta_input_ids, roberta_attention_mask) in zip(bert_loader, roberta_loader):
        bert_input_ids = bert_input_ids.to(device)
        bert_attention_mask = bert_attention_mask.to(device)
        roberta_input_ids = roberta_input_ids.to(device)
        roberta_attention_mask = roberta_attention_mask.to(device)

        # Get model outputs
        bert_logits = bert_model(input_ids=bert_input_ids, attention_mask=bert_attention_mask).logits
        roberta_logits = roberta_model(input_ids=roberta_input_ids, attention_mask=roberta_attention_mask).logits

        # Convert to softmax probabilities
        bert_probs = torch.nn.functional.softmax(bert_logits, dim=1).cpu().numpy()
        roberta_probs = torch.nn.functional.softmax(roberta_logits, dim=1).cpu().numpy()

        bert_probs_all.append(bert_probs)
        roberta_probs_all.append(roberta_probs)

# Concatenate all batches
import numpy as np
bert_probs_all = np.concatenate(bert_probs_all, axis=0)
roberta_probs_all = np.concatenate(roberta_probs_all, axis=0)

# Ensemble prediction
ensemble_probs = (bert_probs_all + roberta_probs_all) / 2
ensemble_preds = np.argmax(ensemble_probs, axis=1)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [31]:
# Evaluate performance
true_labels = test_df["label"].values

acc = accuracy_score(true_labels, ensemble_preds)
f1 = f1_score(true_labels, ensemble_preds, average="macro")

print(f"✅ Ensemble Accuracy: {acc:.4f}")
print(f"✅ Ensemble F1 Macro: {f1:.4f}")


✅ Ensemble Accuracy: 0.8743
✅ Ensemble F1 Macro: 0.8775


In [32]:
print(classification_report(true_labels, ensemble_preds, target_names=unique_labels))

                    precision    recall  f1-score   support

Extremely Negative       0.85      0.92      0.88       911
Extremely Positive       0.88      0.93      0.91      1084
          Negative       0.84      0.85      0.85      1644
           Neutral       0.92      0.86      0.89      1250
          Positive       0.89      0.85      0.87      1855

          accuracy                           0.87      6744
         macro avg       0.87      0.88      0.88      6744
      weighted avg       0.88      0.87      0.87      6744

